__Lab: Pinhole Camera__

In [1]:
import numpy as np

The xy matrix is the 2d representation of the XYZ matrix. Our goal is to find a matrix P which will satisfy:

$XYZ \cdot P \approx xy$

In [2]:
xy = np.array([[ 213.1027,  170.0499], [ 258.1908,  181.3219],
            [ 306.41  ,  193.8464], [ 351.498 ,  183.8268],
            [ 382.8092,  155.6468], [ 411.6155,  130.5978],
            [ 223.7485,  218.2691], [ 267.5841,  230.7935],
            [ 314.5509,  244.5705], [ 357.7603,  235.1771],
            [ 387.819 ,  205.1184], [ 415.3728,  178.1908],
            [ 234.3943,  263.9834], [ 276.9775,  277.1341],
            [ 323.318 ,  291.5372], [ 363.3963,  282.1438],
            [ 392.8288,  251.4589], [ 419.1301,  223.9051]])

XYZ = np.array([[0, -5, 5], [0, -3, 5], [0, -1, 5], [-1, 0, 5],
             [-3, 0, 5], [-5, 0, 5], [0, -5, 3], [0, -3, 3],
             [0, -1, 3], [-1, 0, 3], [-3, 0, 3], [-5, 0, 3],
             [0, -5, 1], [0, -3, 1], [0, -1, 1], [-1, 0, 1],
             [-3, 0, 1], [-5, 0, 1]])

The matrix P should have a shape of 3x4. This is due to XYZ having length 4 (homogeneous coordinates) and XY having length 3, therefore the new matrix P should be 3x4.

We find P by building matrix A (which is given in the syllabus) and using the SVD to find P.

In [3]:
def calibrate(xy, XYZ):
    A = np.empty((0, 12))
    
    for i, (X, Y, Z) in enumerate(XYZ):
        x = xy[i][0]
        y = xy[i][1]
        
        # matrix in lab exercise
        row_1 = np.array([X, Y, Z, 1, 0, 0, 0, 0, -x*X, -x*Y, -x*Z, -x])
        row_2 = np.array([0, 0, 0, 0, X, Y, Z, 1, -y*X, -y*Y, -y*Z, -y])
        
        A = np.vstack((A, row_1))
        A = np.vstack((A, row_2))
        
    _, _, V = np.linalg.svd(A)   
    return V[-1].reshape(3, 4)  

Our matrix P:

In [4]:
P = calibrate(xy, XYZ)
print(P)

[[ -5.29664802e-02   3.69423693e-02  -2.04014448e-02   7.33335654e-01]
 [  1.86948894e-02   4.02549614e-03  -5.91971917e-02   6.73615637e-01]
 [ -5.63200506e-05  -4.07501309e-05  -3.90586158e-05   2.09039106e-03]]


The following functions are used to make our matrices homogenous, and remove the homogenous property. We make a matrix homogenous simply by adding a column of 1s. The homogenous property can be "removed" by removing the last column, returning the result of the division of the new matrix and the last column.

In [5]:
def make_h(matrix):
    ones = np.ones(len(matrix))
    return np.vstack((matrix.T, ones)).T

def rem_h(matrix):
    last_column = matrix[:,-1]
    rest = matrix[:,:-1]
    return rest / last_column[:,None]

def transform(P, XYZ):
    h_XYZ = make_h(XYZ)
    tmp_xy = h_XYZ.dot(P.T)
    return rem_h(tmp_xy)    

Given P, xy and XYZ we can calculate the reprojection error, which is the average of the euclidean distances between the points generated by $p \cdot XYZ$ and the "real" $xy$, since $p \cdot XYZ \approx xy$. 

In [6]:
def reprojection_error(P, xy, XYZ):
    n_xy = transform(P, XYZ)    
    distances = np.linalg.norm(n_xy - xy, axis=1) 
    return np.average(distances)

The average reprojection error is 0.42, which means our $p \cdot XYZ$ is a very good approximation.

In [7]:
avg_re = reprojection_error(P, xy, XYZ)
print(avg_re)

0.421666887358


For the drawing function we first define all the vertices of our unit cube. The order is very important, since some of these vertices have connections between them, and some don't. All the points with connections are defined in "lines".

In [8]:
import matplotlib.pyplot as plt

In [9]:
def draw(P, X, Y, Z, color="blue"):
    cube_3d = np.array([[X, Y, Z], [X+1, Y, Z], [X+1, Y, Z+1], [X, Y, Z+1], 
                        [X, Y+1, Z], [X+1, Y+1, Z], [X+1, Y+1, Z+1], [X, Y+1, Z+1]])
    cube_2d = transform(P, cube_3d)
    
    lines = np.array([[0, 1], [0, 4],
                      [1, 2], [1, 5],
                      [2, 3], [2, 6],
                      [3, 0], [3, 7],
                      [4, 5], [5, 6],
                      [6, 7], [7, 4]])
    
    for p1, p2 in lines:
        x1, y1 = cube_2d[p1]
        x2, y2 = cube_2d[p2]
        plt.plot([x1, x2], [y1, y2], color=color)

As we can see, all the cubes are nicely aligned to the checkerboard. This is due to the fact that the given $XYZ$ and $xy$ (and therefore our P) are choosen in such a way that the coordinate system is perfectly aligned to the checkerboard. In the image we can see what the impact would be of raising your X, your Y or your Z. 

In [11]:
image = plt.imread('./images/calibrationpoints.jpg')
plt.figure(figsize=(15, 15))
plt.imshow(image)
plt.axis('off')

# for x, y, z in XYZ:
#     draw(P, x, y, z)
    
draw(P, 0, 0, 0, color="red")
draw(P, 0, -7, 6, color="red")
draw(P, -6, 0, 4, color="red")

plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'images/calibrationpoints.jpg'